In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

# 📚 Agregaciones, Joins y más

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, when, coalesce, lit

spark = SparkSession.builder.appName("CSVtoDataFrame").getOrCreate()

df = spark.read.csv("/content/libros.csv", header=True, inferSchema=True)
df.show()

+--------------------+--------------------+-----------+-------+-----+----+-----------+
|                Name|              Author|User Rating|Reviews|Price|Year|      Genre|
+--------------------+--------------------+-----------+-------+-----+----+-----------+
|10-Day Green Smoo...|            JJ Smith|        4.7|  17350|    8|2016|Non Fiction|
|   11/22/63: A Novel|        Stephen King|        4.6|   2052|   22|2011|    Fiction|
|12 Rules for Life...|  Jordan B. Peterson|        4.7|  18979|   15|2018|Non Fiction|
|1984 (Signet Clas...|       George Orwell|        4.7|  21424|    6|2017|    Fiction|
|5,000 Awesome Fac...|National Geograph...|        4.8|   7665|   12|2019|Non Fiction|
|A Dance with Drag...| George R. R. Martin|        4.4|  12643|   11|2011|    Fiction|
|A Game of Thrones...| George R. R. Martin|        4.7|  19735|   30|2014|    Fiction|
|A Gentleman in Mo...|         Amor Towles|        4.7|  19699|   15|2017|    Fiction|
|A Higher Loyalty:...|         James Comey|

# Agregaciones (groupBy().agg())

In [3]:
df.groupBy("Author").agg(count("*").alias("total_libros")).show(5)

+--------------+------------+
|        Author|total_libros|
+--------------+------------+
|   Sarah Young|           6|
|    Jill Twiss|           1|
|    Sara Gruen|           1|
|    Doug Lemov|           2|
|Justin Halpern|           1|
+--------------+------------+
only showing top 5 rows


# sum() y avg()

In [4]:
df.agg(
    avg("User Rating").alias("promedio_rating"),
    sum("Reviews").alias("total_reviews")
).show(5)

+-----------------+-------------+
|  promedio_rating|total_reviews|
+-----------------+-------------+
|4.618363636363641|      6574305|
+-----------------+-------------+



In [5]:
df.groupBy("Genre", "Year").agg(
    avg("Price").alias("precio_promedio"),
    sum("Reviews").alias("total_reviews")
).orderBy("Year").show(10)

+-----------+----+------------------+-------------+
|      Genre|Year|   precio_promedio|total_reviews|
+-----------+----+------------------+-------------+
|    Fiction|2009|15.583333333333334|       156824|
|Non Fiction|2009| 15.23076923076923|        78682|
|Non Fiction|2010|              16.0|       105796|
|    Fiction|2010|               9.7|       168185|
|    Fiction|2011|11.619047619047619|       217041|
|Non Fiction|2011|17.620689655172413|       188000|
|    Fiction|2012|12.285714285714286|       417821|
|Non Fiction|2012|17.482758620689655|       236725|
|Non Fiction|2013|18.192307692307693|       175223|
|    Fiction|2013|10.708333333333334|       479684|
+-----------+----+------------------+-------------+
only showing top 10 rows


# Joins

In [6]:
# Crear un DataFrame adicional
data_paises = [("JJ Smith", "EE.UU."), ("Stephen King", "Reino Unido"), ("George Orwell", "Canadá")]
df_paises = spark.createDataFrame(data_paises, ["Author", "Pais"])
df_paises.show()

+-------------+-----------+
|       Author|       Pais|
+-------------+-----------+
|     JJ Smith|     EE.UU.|
| Stephen King|Reino Unido|
|George Orwell|     Canadá|
+-------------+-----------+



In [7]:
df_join = df.join(df_paises, on="Author", how="left")
df_join.select("Author","Pais","Name").show(5)

+--------------------+-----------+--------------------+
|              Author|       Pais|                Name|
+--------------------+-----------+--------------------+
|            JJ Smith|     EE.UU.|10-Day Green Smoo...|
|        Stephen King|Reino Unido|   11/22/63: A Novel|
|       George Orwell|     Canadá|1984 (Signet Clas...|
|National Geograph...|       NULL|5,000 Awesome Fac...|
|  Jordan B. Peterson|       NULL|12 Rules for Life...|
+--------------------+-----------+--------------------+
only showing top 5 rows


# when: condicionales

In [8]:
df.withColumn("categoria_precio", when(col("Price") < 10, "Barato")
                                     .when(col("Price") < 20, "Medio")
                                     .otherwise("Caro")).select("Name", "Price", "categoria_precio").show(5)

+--------------------+-----+----------------+
|                Name|Price|categoria_precio|
+--------------------+-----+----------------+
|10-Day Green Smoo...|    8|          Barato|
|   11/22/63: A Novel|   22|            Caro|
|12 Rules for Life...|   15|           Medio|
|1984 (Signet Clas...|    6|          Barato|
|5,000 Awesome Fac...|   12|           Medio|
+--------------------+-----+----------------+
only showing top 5 rows


# coalesce: reemplaza un nulo

In [9]:
df.withColumn("Reviews_sin_nulos",
              coalesce(col("Reviews"),
              lit(0))).select("Name", "Reviews", "Reviews_sin_nulos").show(5)

+--------------------+-------+-----------------+
|                Name|Reviews|Reviews_sin_nulos|
+--------------------+-------+-----------------+
|10-Day Green Smoo...|  17350|            17350|
|   11/22/63: A Novel|   2052|             2052|
|12 Rules for Life...|  18979|            18979|
|1984 (Signet Clas...|  21424|            21424|
|5,000 Awesome Fac...|   7665|             7665|
+--------------------+-------+-----------------+
only showing top 5 rows


# lit : agrega una columna constante

In [10]:
df.withColumn("Formato", lit("Físico")).select("Name", "Formato").show(5)

+--------------------+-------+
|                Name|Formato|
+--------------------+-------+
|10-Day Green Smoo...| Físico|
|   11/22/63: A Novel| Físico|
|12 Rules for Life...| Físico|
|1984 (Signet Clas...| Físico|
|5,000 Awesome Fac...| Físico|
+--------------------+-------+
only showing top 5 rows
